# Training

In [ ]:
!pip install -Uq fastai
import fastai
from fastai.vision import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path
from fastai.vision.all import get_image_files

pies = "apple_pie", "blueberry_pie", "peach_pie", "pecan_pie", "pi_symbol", "raspberry_pie", "raspberry_pi"
path = Path('/content/drive/MyDrive/pie_classifier')
filenames = get_image_files(path)
len(filenames)

In [ ]:
from fastai.vision.all import ImageDataLoaders, Resize, aug_transforms

def label_func(f): return f.parent.name

dls = ImageDataLoaders.from_folder(path, valid_pct=0.6, vocab=pies, item_tfms=Resize(460), batch_tfms=aug_transforms(size=244))

In [ ]:
dls.show_batch()

In [ ]:
from fastai.vision.all import F1ScoreMulti, vision_learner, resnet50, partial, accuracy_multi

f1_macro = F1ScoreMulti(thresh=0.5, average='macro')
f1_macro.name = 'F1(macro)'
f1_samples = F1ScoreMulti(thresh=0.5, average='samples')
f1_samples.name = 'F1(samples)'
learn = vision_learner(dls, resnet50, metrics=[partial(accuracy_multi, thresh=0.5), f1_macro, f1_samples])

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(2, 3e-2)

In [ ]:
learn.export('/content/drive/MyDrive/pie_classifier/export.pkl')

# Inference

In [ ]:
import requests
from PIL import Image

test_image_url = 'https://bake-eat-repeat.com/wp-content/uploads/2016/05/Raspberry-Pie-3-540x720.jpg.webp'

response = requests.get(test_image_url)
img = Image.open(response.raw)

In [ ]:
from fastai.vision.all import load_learner

inf = load_learner('/content/drive/MyDrive/pie_classifier/export.pkl')
inf.predict(img)